### Web Scrapping -> .doc -> .pdf -> Chunking, Embedding -> VectorDB -> Retreival -> Generation

In [ ]:
# !pip install python-docx
# !pip install docx2pdf

^C


In [1]:
from urllib.parse import urlparse
import requests
from langchain_classic.docstore.document import Document

In [2]:
file_path = 'links.txt'

link_lists = []
with open(file_path, 'r') as file:
    for line in file:
        url = line.strip()
        link_lists.append(url)
        

In [5]:
from bs4 import BeautifulSoup
import re
from docx import Document
from docx2pdf import convert
import os

def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text(separator=' ', strip=True)       # get_text() merges adjacent text, hence separator required

def fetch_webpage(url):
    try:
        response = requests.get(url, verify=False)         # verify=False helps ignoring certifications validity check
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def clean_text(text):
    # Removing extra whitespaces and newlines
    text = ' '.join(text.split())
    # Removing unwanted Characters in url
    text = re.sub(r'http\S+', '',text)
    #Remove special characters except punctuations
    text = re.sub(r'[^a-zA-Z0-9\s.,!?\'"@-]+', '', text)     # Improved @, - for no email issues
    return text


base_dir = os.getcwd()

docx_folder = os.path.join(base_dir, "data_docx")
pdf_folder = os.path.join(base_dir, "data_pdf")

os.makedirs(docx_folder, exist_ok=True)
os.makedirs(pdf_folder, exist_ok=True)


def write_to_docx(text_content, output_filename):
    doc = Document()
    doc.add_heading(output_filename.split('.')[0], level=1)
    doc.add_paragraph(text_content)

    docx_path = os.path.join(docx_folder, output_filename)
    doc.save(docx_path)
    return docx_path

def docx_to_pdf(docx_path, pdf_filename):
    pdf_path = os.path.join(pdf_folder, pdf_filename)
    try:
        convert(docx_path, pdf_path)
        print(f"Successfully Converted {docx_path}")
    except Exception as e:
        print(f"Error Converting {docx_path}: {e}")



In [ ]:
from pprint import pprint

for url in link_lists:
    html_content = fetch_webpage(url)
    page_text = parse_html(html_content)
    final_text = clean_text(page_text)
    output_docx_filename = url.split('/')[-1] + '.docx'
    output_pdf_filename = url.split('/')[-1] + '.pdf'
    
    docx_path = write_to_docx(final_text, output_docx_filename)
    docx_to_pdf(docx_path,output_pdf_filename) 

In [ ]:
file_name = "https://www.svnit.ac.in/web/department/physics/contact_us.php".split('/')[-1] + '.docx'


'contact_us'